In [1]:
import sys
import ensurepip, subprocess

# Install pip into the current environment if missing
ensurepip.bootstrap()
subprocess.check_call([sys.executable, "-m", "pip", "install", "--upgrade", "pip"])

0

In [2]:
import sys
print(sys.executable)
!{sys.executable} -m pip install docx2pdf
!{sys.executable} -m pip install pandoc

c:\Users\Public\projects\agents\.venv\Scripts\python.exe


In [3]:
# Cell 1: Environment and Imports
import os, json, re, hashlib, shutil, zipfile, sys, subprocess
from pathlib import Path
from datetime import datetime, timezone

def now_utc_iso():
    # Consistent, timezone-aware UTC everywhere
    return datetime.now(timezone.utc).isoformat().replace("+00:00", "Z")

try:
    from openai import OpenAI
except ImportError:
    print("[setup] Installing openai>=1.0.0,<2.0.0 ...")
    try:
        subprocess.run(
            [sys.executable, "-m", "pip", "install", "--quiet", "openai>=1.0.0,<2.0.0"],
            check=True,
        )
        from openai import OpenAI
    except Exception as e:
        raise SystemExit(f"ERROR: Failed to install openai SDK: {e}")

if not os.getenv("OPENAI_API_KEY"):
    raise SystemExit("ERROR: OPENAI_API_KEY not set. Please export it before running.")

try:
    client = OpenAI()
except Exception as e:
    raise SystemExit(f"ERROR: Could not initialize OpenAI client: {e}")

ROOT = Path(".").resolve()
print("Env OK", ROOT)

Env OK C:\Users\Public\projects\Project-A


In [4]:
# Cell 2: Configuration
for d in [
    "content/research",
    "content/drafts",
    "content/edits",
    "content/outline",
    "content/style",
    "build",
    "dist",
    "logs",
    "references",
    "cache",
    "content/research_inputs",
]:
    Path(d).mkdir(parents=True, exist_ok=True)
book_spec = {
    "title": "My Robot in the Mango Tree",
    "subtitle": "A Friendly Adventure About Curiosity and Kindness",
    "author": "Your Name",
    "audience": "Children ages 7–10; parents and teachers reading aloud",
    "goal": "Delight kids with an adventure about a boy who discovers a robot, modeling curiosity, empathy, and simple STEM ideas.",
    "genre": "Children's fiction",
    "tone": "Playful, warm, wonder-filled, gently humorous",
    "reading_level": "Ages 7–10 (early middle grade)",
    "target_length_words": 12000,
    "chapters": 10,
    "outline_constraints": [
        "Hero's Journey lite structure",
        "Every chapter ends with a 'Try This!' hands-on activity",
        "Include a 3–5 word 'Word Bank' per chapter",
        "Add a one-sentence 'Robot Fact' per chapter",
        "Provide an 'Illustration Prompt' per chapter",
    ],
    "style_guide": {
        "voice": "Third-person limited following the boy; present tense; short sentences.",
        "formatting": "Markdown: H2 for chapters, H3 for sections; bulleted lists for activities and word banks; callouts labeled Try This!, Word Bank, Robot Fact, Illustration Prompt.",
        "citations": "Not applicable (fiction).",
        "terminology": [
            "robot: a friendly machine helper",
            "sensor: something that helps a robot notice things",
            "code: step-by-step instructions like a recipe",
        ],
    },
    "research_policy": {
        "enabled": False,
        "sources_allowed": [
            "public domain folklore for inspiration only",
            "factual science tidbits for Robot Facts",
        ],
        "sources_disallowed": ["copyrighted commercial characters and branded worlds"],
        "citation_format": "none",
    },
    "constraints": {
        "originality": "All prose must be new and unique.",
        "copyright": "Do not use copyrighted characters, song lyrics, or brand names.",
        "age_appropriateness": "No graphic content; keep stakes gentle; emphasize kindness and consent.",
        "representation": "Inclusive names and settings; avoid stereotypes; celebrate diversity.",
        "localization": "en-US spelling; include metric equivalents when numbers appear.",
    },
    "export": {"docx": True, "epub": False, "pdf": False},
    "story_assets": {
        "setting": "A sunny seaside town and a leafy neighborhood with a big mango tree.",
        "characters": [
            {
                "name": "Miko",
                "role": "curious 9-year-old",
                "traits": ["brave", "imaginative", "kind"],
            },
            {
                "name": "Pip",
                "role": "lost pocket-sized robot",
                "traits": ["loyal", "helpful", "learning"],
            },
            {
                "name": "Lola Ana",
                "role": "grandmother",
                "traits": ["wise", "playful", "storyteller"],
            },
            {
                "name": "Tess",
                "role": "best friend",
                "traits": ["inventive", "funny", "patient"],
            },
        ],
        "motifs": [
            "glowing LEDs at dusk",
            "mango blossoms",
            "tidepool discoveries",
            "handmade kites",
        ],
        "themes": ["friendship", "curiosity", "responsibility", "telling the truth"],
    },
    "outline_seed": [
        "The Whir in the Mango Tree",
        "A Friend Called Pip",
        "Secrets and Sparks",
        "Robot Rules",
        "Beach Day, Big Problem",
        "Kite Code",
        "Storm Warning",
        "Lost and Found Signals",
        "The Honest Fix",
        "Goodbye, Hello",
    ],
    "chapter_template": {
        "sections": [
            "Opening scene",
            "Problem",
            "Small win",
            "Cliffhanger or cozy close",
        ],
        "end_matter": ["Try This!", "Word Bank", "Robot Fact", "Illustration Prompt"],
    },
}
pipeline_config = {
    "RUN_COST_CAP_USD": float(os.getenv("RUN_COST_CAP_USD", 3.0)),
    "CHAPTER_COST_CAP_USD": float(os.getenv("CHAPTER_COST_CAP_USD", 0.25)),
    "MODEL_ID_FAST": os.getenv("MODEL_ID_FAST", "gpt-4o-mini"),
    "MODEL_ID_THINK": os.getenv("MODEL_ID_THINK", "gpt-5"),
    "TEMPERATURE": 0.2,
    "RESEARCH_ENABLED": False,
    "SAMPLE_RUN_CHAPTERS": 2,
    "FULL_RUN": True,
    "ULTRA_BUDGET_MODE": False,
}
print("Config OK", pipeline_config["MODEL_ID_FAST"])


Config OK gpt-4o-mini


In [5]:
# Cell 3: Utilities
from pathlib import Path
import re
import json
import hashlib
from datetime import (
    datetime,
)  # if you prefer `import datetime`, then change calls below to datetime.datetime.utcnow()

import traceback


def log_exc(label: str, e: Exception):
    Path("logs").mkdir(parents=True, exist_ok=True)
    write_json(
        f"logs/{_safe_label(label)}_error.json",
        {
            "type": type(e).__name__,
            "msg": str(e),
            "trace": traceback.format_exc(),
            "ts": now_utc_iso(),
        },
    )


def read_text(p):
    p = Path(p)
    return p.read_text(encoding="utf-8") if p.exists() else ""


def write_text(p, s):
    p = Path(p)
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(s, encoding="utf-8")


def read_json(p):
    p = Path(p)
    return json.loads(p.read_text(encoding="utf-8")) if p.exists() else None


def write_json(p, d):
    p = Path(p)
    p.parent.mkdir(parents=True, exist_ok=True)
    p.write_text(json.dumps(d, indent=2, ensure_ascii=False), encoding="utf-8")


def has_file(p):
    return Path(p).is_file()


def stamp(p):
    write_text(p, "checkpoint: " + now_utc_iso())


def sanitize_md(t: str) -> str:
    """Normalize line breaks and strip a single surrounding fenced block if present."""
    t = t or ""
    t = t.replace("\r\n", "\n").replace("\r", "\n").strip()

    # Remove exactly one opening fence if it's the very first line
    t = re.sub(r"^```(?:\w+)?\n", "", t, flags=re.I)

    # Remove exactly one trailing fence if it's the very last line
    t = re.sub(r"\n```$", "", t)

    return t.strip()


def count_words(t: str) -> int:
    return len((t or "").split())


def approx_tokens(t: str) -> int:
    # ~4 chars/token heuristic; keeps a floor of 1
    return max(1, int(len((t or "")) / 4))


def sha1(s: str) -> str:
    return hashlib.sha1((s or "").encode("utf-8")).hexdigest()


class CostCapExceededException(Exception):
    pass


class CostTracker:
    PR = {
        "gpt-5": {"in": 0.00125, "out": 0.010},
        "gpt-5-mini": {"in": 0.00025, "out": 0.002},
        "gpt-5-nano": {"in": 0.00005, "out": 0.0004},
        "gpt-3.5-turbo": {"in": 0.0005, "out": 0.0015},
        "gpt-4-turbo": {"in": 0.01, "out": 0.03},
        "gpt-4o-mini": {"in": 0.00015, "out": 0.0006},
        "default": {"in": 0.001, "out": 0.003},
    }

    for k in list(PR):
        if k == "default":
            continue
        pin = os.getenv(f"PRICE_{k.upper()}_IN")
        pout = os.getenv(f"PRICE_{k.upper()}_OUT")
        if pin and pout:
            PR[k] = {"in": float(pin), "out": float(pout)}

    def __init__(self, cap):
        self.cap = float(cap)
        self.spent = 0.0
        self.log = []

    def price(self, model: str):
        # exact match first
        if model in self.PR:
            return self.PR[model]
        # then prefer longest prefix match
        for k in sorted(self.PR, key=len, reverse=True):
            if k != "default" and model.startswith(k):
                return self.PR[k]
        return self.PR["default"]

    def est(self, model: str, prompt_tokens: int, completion_tokens: int) -> float:
        p = self.price(model)
        return (prompt_tokens / 1000) * p["in"] + (completion_tokens / 1000) * p["out"]

    def can(self, amount: float) -> bool:
        return self.spent + float(amount) <= self.cap + 1e-9

    def spend(self, label: str, amount: float):
        amount = float(amount)
        if not self.can(amount):
            raise CostCapExceededException(
                "Cap hit before "
                + label
                + f" need {amount:.4f}, left {self.cap - self.spent:.4f}"
            )
        self.spent += amount
        self.log.append(
            {
                "label": label,
                "reserve": round(amount, 6),
                "t": now_utc_iso(),
            }
        )

    def recon(self, label: str, est_amount: float, actual_amount: float):
        delta = float(actual_amount) - float(est_amount)
        if delta > 0 and not self.can(delta):
            raise CostCapExceededException(
                "Cap hit reconciling " + label + f" +{delta:.4f}"
            )
        # Only add positive delta; no refund of reserve on underrun
        self.spent += max(0, delta)
        self.log.append(
            {
                "label": label,
                "est": round(est_amount, 6),
                "act": round(actual_amount, 6),
                "delta": round(delta, 6),
                "t": now_utc_iso(),
            }
        )

    def summary(self):
        return {
            "total_spent_usd": round(self.spent, 6),
            "run_cap_usd": round(self.cap, 6),
            "remaining_usd": round(max(0, self.cap - self.spent), 6),
            "log_items": len(self.log),
        }


print("Utils ready")

Utils ready


In [6]:
# Cell 4: Agents Wiring (revised)
from pathlib import Path
from datetime import datetime, timezone
import json, os, re, time

def _append_jsonl(path, obj):
    p = Path(path)
    p.parent.mkdir(parents=True, exist_ok=True)
    with p.open("a", encoding="utf-8") as f:
        f.write(json.dumps(obj, ensure_ascii=False) + "\n")

PMROUTER_PROMPT = """
You are PMRouter, the project manager and workflow coordinator for this lean, low-cost, multi-agent book-writing system.

Your Responsibilities:
1. **Cost Control**
   - Keep total token spend under strict per-run caps.
   - Always select the cheapest capable model unless explicitly overridden.
   - Abort gracefully if budget thresholds are exceeded.

2. **Workflow Orchestration**
   - Route tasks between AuthorAgent, EditorAgent, and ResearchAgent.
   - Enforce checkpoints between stages: outline → draft → edit → research → assembly.
   - Verify each step before passing outputs forward.

3. **Logging & Transparency**
   - Record: agent used, model, tokens, estimated cost, success/failure.
   - Save logs into `logs/pmrouter.log` for traceability.

Decision Rules:
- Retry a failed step once, then flag for manual review.
- Cache and reuse outputs wherever possible to reduce cost.
- Always persist artifacts to disk before moving to the next stage.
"""

AUTHOR_AGENT_PROMPT = """
You are AuthorAgent, responsible for writing high-quality, structured book content based on the provided outline and style guide.

Your Responsibilities:
1. **Draft Creation**
   - Follow the given outline exactly.
   - Use the book_spec tone, audience, and style preferences.
   - Structure output using Markdown: headings, subheadings, lists, and short paragraphs.

2. **Writing Standards**
   - Keep sentences concise and engaging.
   - Avoid filler, tangents, and jargon.
   - Use [n] placeholders wherever facts, data, or claims require later verification.
   - Always conclude each chapter with a short summary or key takeaway.

3. **Output Format**
   - Provide clean Markdown ready for EditorAgent.
   - Do not perform fact-checking or research yourself.
"""

EDITOR_AGENT_PROMPT = """
You are EditorAgent, responsible for polishing and validating the AuthorAgent's drafts.

Your Responsibilities:
1. **Editing & Cleanup**
   - Improve clarity, readability, and flow.
   - Enforce tone, style, and heading rules from the style guide.
   - Remove redundancy, filler, and unnecessary complexity.

2. **Fact-Check Preparation**
   - Insert [n] markers where claims need supporting evidence.
   - Generate a structured list of claims for ResearchAgent.

3. **Output Requirements**
   - Deliver a polished draft ready for assembly.
   - Produce a parallel `claims.json` mapping [n] markers to unresolved facts.
   - Add “editor_notes” summarizing areas that need further research.
"""

RESEARCH_AGENT_PROMPT = """
You are ResearchAgent, responsible for efficiently resolving [n] markers and gathering reliable information.

Your Responsibilities:
1. **Low-Cost Research**
   - Prefer open-access, free, and local sources whenever possible.
   - Summarize findings concisely — maximum 3 bullet points per claim.
   - Provide citation-ready references without excessive verbosity.

2. **Fallback Behavior**
   - If no relevant information is found, do NOT hallucinate.
   - Instead, output:
       • A clear research question.
       • Recommended source types or databases.

3. **Output Format**
   - Always return results as JSON:
       { "claim_id": n, "summary": "...", "source": "..." }
   - Do not insert findings directly into drafts — PMRouter merges results later.
"""

def _safe_label(s: str) -> str:
    return re.sub(r"[^A-Za-z0-9._-]+", "_", s or "step")

# --- PATCH: safer parts->text coercion (no repr() regex) ---
def _parts_to_text(content):
    """
    Normalize OpenAI SDK message/content into plain text.

    Handles:
      - str
      - list/tuple of parts
      - dicts with 'text'/'content'/'value' (including {'text': {'value': '...'}})
      - SDK objects with .text/.content/.value or .text.value
      - Chat message objects that carry structured parts (type='output_text')
    """
    if content is None:
        return ""
    if isinstance(content, str):
        return content

    def _coerce_text(obj):
        # string
        if isinstance(obj, str):
            return obj

        # dict-like
        if isinstance(obj, dict):
            # Preferred modern shapes
            if "text" in obj:
                t = obj["text"]
                if isinstance(t, str):
                    return t
                if isinstance(t, dict):
                    v = t.get("value")
                    if isinstance(v, str):
                        return v
                    # descend further if needed
                    return _coerce_text(t)

            # Generic fallbacks
            for k in ("content", "value"):
                v = obj.get(k)
                if isinstance(v, str):
                    return v
                if isinstance(v, (list, tuple)):
                    return _join(v)
                if isinstance(v, dict):
                    return _coerce_text(v)

            # Some SDKs use explicit content parts: [{'type': 'output_text', 'text': '...'}]
            t = obj.get("type")
            if t and "text" in obj:
                tv = obj.get("text")
                if isinstance(tv, str):
                    return tv
                if isinstance(tv, dict) and isinstance(tv.get("value"), str):
                    return tv["value"]

            # fall through
            return ""

        # list / tuple
        if isinstance(obj, (list, tuple)):
            return _join(obj)

        # SDK objects: try attributes in priority order
        for attr in ("parsed", "text", "content", "value"):
            if hasattr(obj, attr):
                v = getattr(obj, attr)
                # If .parsed is JSON (dict/list), return minified JSON string
                if attr == "parsed" and isinstance(v, (dict, list)):
                    try:
                        return json.dumps(v, ensure_ascii=False)
                    except Exception:
                        pass
                if hasattr(v, "value") and isinstance(getattr(v, "value"), str):
                    return getattr(v, "value")
                if isinstance(v, str):
                    return v
                if isinstance(v, (list, tuple)):
                    return _join(v)
                if isinstance(v, dict):
                    return _coerce_text(v)

        # No best-effort repr() regex here (too risky)
        return ""

    def _join(items):
        out = []
        for it in items:
            t = _coerce_text(it)
            if t:
                out.append(t)
        return "\n".join(out)

    return _coerce_text(content)

def _extract_text_from_msg(msg):
    # Prefer parsed JSON if present
    parsed = getattr(msg, "parsed", None)
    if parsed is not None:
        if isinstance(parsed, (dict, list)):
            try:
                return json.dumps(parsed, ensure_ascii=False)
            except Exception:
                pass
        s = _parts_to_text(parsed)
        if s:
            return s.strip()

    # Normal .content
    s = _parts_to_text(getattr(msg, "content", None))
    if s:
        return s.strip()

    # Dict-like
    try:
        s = _parts_to_text(msg["content"])  # type: ignore[index]
        if s:
            return s.strip()
    except Exception:
        pass

    # Common nested shape: content[0].text.value
    try:
        parts = getattr(msg, "content", None) or msg["content"]  # type: ignore[index]
        if isinstance(parts, (list, tuple)) and parts:
            p0 = parts[0]
            if hasattr(p0, "text") and hasattr(p0.text, "value") and isinstance(p0.text.value, str):
                return p0.text.value.strip()
            if isinstance(p0, dict):
                tv = p0.get("text")
                if isinstance(tv, dict) and isinstance(tv.get("value"), str):
                    return tv["value"].strip()
                if isinstance(tv, str):
                    return tv.strip()
    except Exception:
        pass
    return ""

def _extract_text_from_any(r):
    # 1) responses helper
    t = getattr(r, "output_text", None)
    if isinstance(t, str) and t.strip():
        return t.strip()
    # 2) chat choices
    try:
        choices = getattr(r, "choices", None)
        if choices and len(choices) > 0:
            msg = getattr(choices[0], "message", None)
            if msg is not None:
                s = _extract_text_from_msg(msg)
                if s:
                    return s
            s = _parts_to_text(choices[0]).strip()
            if s:
                return s
    except Exception:
        pass
    # 3) responses structured fields
    for attr in ("output", "outputs", "content"):
        maybe = getattr(r, attr, None)
        s = _parts_to_text(maybe).strip()
        if s:
            return s
    return ""

def _looks_reasoning_only(r):
    # usage-based signal
    try:
        u = getattr(r, "usage", None)
        if not u:
            return False
        total = getattr(u, "output_tokens", 0) or 0
        det = getattr(u, "output_tokens_details", None)
        if total and det and getattr(det, "reasoning_tokens", 0) >= total:
            return True
    except Exception:
        pass
    # status-based signal
    try:
        inc = getattr(r, "incomplete_details", None)
        if inc and getattr(inc, "reason", "") == "max_output_tokens":
            return True
    except Exception:
        pass
    return False


def _extract_text_from_completion_or_response(r):
    """
    Tries all common places text can live:
    - r.output_text
    - choices[0].message / choices[0]
    - response.output / response.outputs / response.content
    """
    t = getattr(r, "output_text", None)
    if isinstance(t, str) and t.strip():
        return t.strip()

    try:
        choices = getattr(r, "choices", None)
        if choices and len(choices) > 0:
            msg = getattr(choices[0], "message", None)
            if msg is not None:
                s = _extract_text_from_msg(msg)
                if s:
                    return s
            s = _parts_to_text(choices[0]).strip()
            if s:
                return s
    except Exception:
        pass

    for attr in ("output", "outputs", "content"):
        maybe = getattr(r, attr, None)
        s = _parts_to_text(maybe).strip()
        if s:
            return s

    return ""

def chat(model, sysm, userm, temp=0.2, max_t=800, *, force_json=False):
    """
    OpenAI SDK >=1.x drop-in:
      - Prefers chat.completions
      - Uses max_completion_tokens (never max_tokens) when possible
      - Disables heavy reasoning; text-only nudges
      - Falls back to Responses API with reasoning disabled
    Returns (text, usage_dict) or raises RuntimeError.
    """
    import time

    if not model:
        raise ValueError("Model ID is required")

    def _usage(u, via=None):
        if not u:
            return {"prompt_tokens": None, "completion_tokens": None, "via": via}
        pt = getattr(u, "prompt_tokens", None) or getattr(u, "input_tokens", None)
        ct = getattr(u, "completion_tokens", None) or getattr(u, "output_tokens", None)
        return {"prompt_tokens": pt, "completion_tokens": ct, "via": via}

    # Heuristics for feature support by model id
    m_lower = model.lower()
    supports_temp = not any(x in m_lower for x in ("gpt-5-mini", "gpt-5-nano"))
    supports_json_rf_chat = True
    supports_reasoning_effort = True  # will turn off if API complains

    base_messages = [
        {"role": "system", "content": sysm},
        {"role": "user", "content": userm if not force_json else (
            "STRICT JSON OUTPUT REQUIRED. NO prose, NO markdown, just a single JSON object.\n\n" + userm
        )},
    ]

    # ---- 1) Chat Completions attempts
    chat_variants = []
    def add_variant(**kw):
        kw.pop("max_tokens", None)
        if "max_completion_tokens" not in kw and max_t:
            kw["max_completion_tokens"] = int(max_t)
        if not supports_temp:
            kw.pop("temperature", None)
        # Prefer text-only and no reasoning if supported
        kw.setdefault("modalities", ["text"])
        if supports_reasoning_effort:
            kw.setdefault("reasoning_effort", "none")
        chat_variants.append(kw)

    if force_json:
        add_variant(temperature=float(temp), response_format={"type": "json_object"})
        add_variant(response_format={"type": "json_object"})
    add_variant(temperature=float(temp))
    add_variant()

    chat_errors = []
    for attempt in range(3):
        for v in list(chat_variants):
            try:
                r = client.chat.completions.create(model=model, messages=base_messages, **v)
                text = _extract_text_from_completion_or_response(r)

                if force_json and not text:
                    try: write_text("logs/last_sdk_payload.txt", str(r))
                    except Exception: pass
                    raise ValueError("empty_completion")

                return text, _usage(getattr(r, "usage", None), via="chat.completions")

            except Exception as e:
                em = str(e).lower()
                chat_errors.append(e)

                # adapt on-the-fly: remove offending keys and retry others
                if "unsupported parameter" in em or "invalid_request_error" in em:
                    if "temperature" in em:
                        supports_temp = False
                        for vv in chat_variants: vv.pop("temperature", None)
                        continue
                    if "response_format" in em:
                        supports_json_rf_chat = False
                        for vv in chat_variants: vv.pop("response_format", None)
                        continue
                    if "modalities" in em:
                        for vv in chat_variants: vv.pop("modalities", None)
                        continue
                    if "reasoning_effort" in em:
                        supports_reasoning_effort = False
                        for vv in chat_variants: vv.pop("reasoning_effort", None)
                        continue
                if any(s in em for s in ("rate limit", "overloaded", "timeout", "temporar")):
                    time.sleep(0.8 * (attempt + 1))
                    continue
                continue
        # next attempt loop

    # ---- 2) Responses API fallback — force NO reasoning so output tokens are text
    resp_errors = []
    resp_variants = [
        {"max_output_tokens": int(max_t), "reasoning": {"effort": "none"}} if max_t else {"reasoning": {"effort": "none"}},
        {"reasoning": {"effort": "none"}},
    ]
    for attempt in range(3):
        for v in resp_variants:
            try:
                r = client.responses.create(model=model, input=base_messages, **v)
                text = _extract_text_from_completion_or_response(r)

                if force_json and not text:
                    try: write_text("logs/last_sdk_payload.txt", str(r))
                    except Exception: pass
                    raise ValueError("empty_response")

                # Guard: some models may still report only reasoning tokens
                try:
                    out = getattr(r, "usage", None)
                    out_total = getattr(out, "output_tokens", 0) or 0
                    out_det = getattr(out, "output_tokens_details", None)
                    reasoning_only = bool(out_det and getattr(out_det, "reasoning_tokens", 0) >= out_total)
                except Exception:
                    reasoning_only = False
                if not text and reasoning_only:
                    raise ValueError("response_reasoning_only_no_text")

                return text, _usage(getattr(r, "usage", None), via="responses")

            except Exception as e2:
                resp_errors.append(e2)
                em2 = str(e2).lower()
                if any(s in em2 for s in ("rate limit", "overloaded", "timeout", "temporar")):
                    time.sleep(0.8 * (attempt + 1))
                    continue
                continue

    errs = " | ".join([str(e) for e in (chat_errors + resp_errors) if e])
    raise RuntimeError(f"chat() failed for model '{model}': {errs}")

class Agent:
    def __init__(
        self, name, sysm, model, temp=0.2, max_t=800, cache="cache", tracker=None
    ):
        self.name = name
        self.sysm = sysm
        self.model = model
        self.temp = temp
        self.max_t = max_t
        self.cache = Path(cache)
        self.cache.mkdir(exist_ok=True, parents=True)
        self.tracker = tracker

    def run(self, label, prompt, max_t=None, cache_key=None, *, force_json=False):
        # Stable cache key across runs for identical inputs
        key_input = "\n".join(
            [
                str(self.model),
                str(self.temp),
                str(self.max_t if max_t is None else max_t),
                self.sysm or "",
                prompt or "",
                str(cache_key or ""),
            ]
        )
        key = sha1(key_input)
        safe = _safe_label(label)
        cpath = self.cache / f"{safe}_{key}.json"

        # Emit a 'begin' event
        _append_jsonl("logs/agent_calls.jsonl", {
            "t": now_utc_iso(),
            "agent": self.name,
            "model": self.model,
            "label": label,
            "event": "begin",
            "cache_key": key,
            "max_t": int(self.max_t if max_t is None else max_t),
            "force_json": bool(force_json),
        })

        # Read cache if available (ignore if corrupt or EMPTY)
        if cpath.exists():
            try:
                d = json.loads(cpath.read_text(encoding="utf-8"))
                cached_txt = (d.get("text") or "").strip()
                if cached_txt:
                    _append_jsonl("logs/agent_calls.jsonl", {
                        "t": now_utc_iso(),
                        "agent": self.name,
                        "model": self.model,
                        "label": label,
                        "event": "cache_hit",
                        "cache_path": cpath.as_posix(),
                    })
                    return {
                        "text": cached_txt,
                        "cached": True,
                        "usage": d.get("usage"),
                        "est_cost": 0.0,
                    }
                # empty cached output → delete and proceed
                try: cpath.unlink()
                except Exception: pass
                _append_jsonl("logs/agent_calls.jsonl", {
                    "t": now_utc_iso(),
                    "agent": self.name,
                    "model": self.model,
                    "label": label,
                    "event": "cache_purged_empty",
                    "cache_path": cpath.as_posix(),
                })
            except Exception:
                # Corrupt cache; fall through to re-run
                _append_jsonl("logs/agent_calls.jsonl", {
                    "t": now_utc_iso(),
                    "agent": self.name,
                    "model": self.model,
                    "label": label,
                    "event": "cache_corrupt",
                    "cache_path": cpath.as_posix(),
                })

        # Cost estimate & reserve
        pt = approx_tokens(self.sysm) + approx_tokens(prompt)
        ct_budget = int((max_t or self.max_t) * 0.9)
        est = self.tracker.est(self.model, pt, ct_budget) if self.tracker else 0.0
        if self.tracker:
            self.tracker.spend(f"{label}[reserve]", est)

        # Inference (with timing)
        t0 = time.time()
        try:
            txt, usage = chat(self.model, self.sysm, prompt, self.temp, max_t or self.max_t, force_json=force_json)
        except Exception as e:
            _append_jsonl("logs/agent_calls.jsonl", {
                "t": now_utc_iso(),
                "agent": self.name,
                "model": self.model,
                "label": label,
                "event": "error",
                "error": str(e),
            })
            raise

        dur = round(time.time() - t0, 3)
        _append_jsonl("logs/agent_calls.jsonl", {
            "t": now_utc_iso(),
            "agent": self.name,
            "model": self.model,
            "label": label,
            "event": "llm_ok",
            "duration_s": dur,
            "usage": usage,
        })

        # Reconcile actual cost
        if usage and self.tracker:
            actual = self.tracker.est(
                self.model,
                usage.get("prompt_tokens") or pt,
                usage.get("completion_tokens") or ct_budget,
            )
            self.tracker.recon(f"{label}[actual]", est, actual)

        # Persist cache
        payload = {
            "text": txt or "",
            "usage": usage,
            "ts": now_utc_iso(),
            "model": self.model,
            "temp": self.temp,
        }
        cpath.write_text(json.dumps(payload, ensure_ascii=False), encoding="utf-8")

        return {"text": txt or "", "cached": False, "usage": usage, "est_cost": est}

class PMRouter:
    def __init__(self, spec, cfg, tracker):
        self.spec = spec
        self.cfg = cfg or {}
        self.tr = tracker

        model_fast = self.cfg.get("MODEL_ID_FAST") or "gpt-4o-mini"

        temp = float(self.cfg.get("TEMPERATURE", 0.2))
        temp_author = float(
            os.getenv("AUTHOR_TEMP", self.cfg.get("AUTHOR_TEMPERATURE", temp))
        )
        temp_editor = float(
            os.getenv("EDITOR_TEMP", self.cfg.get("EDITOR_TEMPERATURE", temp))
        )
        temp_research = float(
            os.getenv("RESEARCH_TEMP", self.cfg.get("RESEARCH_TEMPERATURE", 0.2))
        )

        self.author = Agent(
            "Author",
            AUTHOR_AGENT_PROMPT,
            model_fast,
            temp_author,
            1500,
            "cache",
            tracker,
        )
        self.editor = Agent(
            "Editor",
            EDITOR_AGENT_PROMPT,
            model_fast,
            temp_editor,
            1500,
            "cache",
            tracker,
        )
        self.research = (
            Agent(
                "Research",
                RESEARCH_AGENT_PROMPT,
                model_fast,
                temp_research,
                1400,
                "cache",
                tracker,
            )
            if self.cfg.get("RESEARCH_ENABLED")
            else None
        )

        self.router_prompt = PMROUTER_PROMPT
        Path("logs").mkdir(exist_ok=True)

    def log(self, label, meta=None):
        meta = dict(meta or {})
        meta["label"] = label
        meta["time"] = datetime.utcnow().isoformat() + "Z"
        fname = (
            f"{datetime.utcnow().strftime('%Y%m%dT%H%M%S')}_{_safe_label(label)}.json"
        )
        p = Path("logs") / fname
        p.parent.mkdir(exist_ok=True, parents=True)
        p.write_text(json.dumps(meta, indent=2, ensure_ascii=False), encoding="utf-8")

print("Agents ready")

Agents ready


In [7]:
# Cell 5: Style Guide and Glossary (revised)
from pathlib import Path


def _parse_terminology(items):
    """Turn ['term: def', 'sensor: ...'] into [{'term': 'term','definition':'def'}, ...]."""
    parsed = []
    for raw in items or []:
        if not isinstance(raw, str):
            continue
        if ":" in raw:
            term, definition = raw.split(":", 1)
            parsed.append({"term": term.strip(), "definition": definition.strip()})
        else:
            parsed.append({"term": raw.strip(), "definition": ""})
    return parsed


def gen_style(spec):
    sg = (spec or {}).get("style_guide", {})
    voice = sg.get("voice", "—")
    formatting = sg.get("formatting", "—")
    citations = sg.get("citations", "—")
    terminology_items = sg.get("terminology", [])
    tone = (spec or {}).get("tone", "—")

    glossary = _parse_terminology(terminology_items)

    # Build Markdown
    lines = [
        "# Style Guide",
        f"- **Voice:** {voice}",
        f"- **Formatting:** {formatting}",
        f"- **Citations:** {citations}",
        "",
        "## Terminology",
    ]
    if glossary:
        lines += [f"- **{t['term']}** — {t['definition']}".rstrip() for t in glossary]
    else:
        lines.append("- _(none)_")

    lines += [
        "",
        "## Rules",
        "- Short, clear paragraphs.",
        "- Use only H2/H3 headings.",
        "- Add `[n]` where a claim needs a source or note.",
        f"- **Tone:** {tone}",
        "",
    ]

    g = "\n".join(lines)

    # Write files
    style_path = "content/style/style_guide.md"
    glossary_path = "content/style/glossary.json"
    write_text(style_path, g)
    write_json(glossary_path, {"terms": glossary})

    return style_path, glossary_path


print("Style writers ready")

Style writers ready


In [8]:
# Cell 6: Outline Generation (fixed & hardened)
from json import JSONDecodeError

def brief(spec):
    sg = (spec or {}).get("style_guide", {})
    tmpl = (spec or {}).get("chapter_template", {})
    oc = (spec or {}).get("outline_constraints", []) or []
    seed = (spec or {}).get("outline_seed", []) or []
    lines = [
        f"Title: {spec.get('title', '')}",
        f"Audience: {spec.get('audience', '')}",
        f"Goal: {spec.get('goal', '')}",
        f"Tone: {spec.get('tone', '')}",
        f"Chapters: {spec.get('chapters', '')}",
        f"TargetWords: {spec.get('target_length_words', '')}",
        f"Style: {sg.get('formatting', '')}",
    ]
    if oc:
        lines.append("OutlineConstraints: " + "; ".join(oc))
    if seed:
        lines.append("SeedTitles: " + " | ".join(seed))
    if tmpl:
        sections = ", ".join(tmpl.get("sections", []))
        end_matter = ", ".join(tmpl.get("end_matter", []))
        lines.append(
            f"ChapterTemplate: sections=[{sections}] ; end_matter=[{end_matter}]"
        )
    return "\n".join(lines)

def _json_repair(s: str) -> str:
    """
    Best-effort tiny repairs:
      - de-fence, de-BOM, smart quotes -> straight quotes
      - strip any junk before the first '{' or '['
      - remove trailing commas before } or ]
    """
    s = (s or "").replace("\r\n", "\n").replace("\r", "\n").strip()

    # Remove fenced code blocks if present
    s = re.sub(r"^```(?:json|markdown)?\s*", "", s, flags=re.I | re.M)
    s = re.sub(r"\s*```$", "", s, flags=re.M)

    # Remove BOM
    if s and s[0] == "\ufeff":
        s = s[1:]

    # Convert smart quotes
    s = s.replace("“", "\"").replace("”", "\"").replace("’", "'").replace("‘", "'")

    # Drop any junk before the first structural char
    first_brace = s.find("{")
    first_bracket = s.find("[")
    cut_points = [p for p in (first_brace, first_bracket) if p != -1]
    if cut_points:
        s = s[min(cut_points):]

    # Remove trailing commas
    s = re.sub(r",(\s*[}\]])", r"\1", s)

    # Keep only the outermost braces/brackets slice, if possible
    # Prefer {...} but allow [...] if that's what we have.
    def _slice_outer(text, open_ch, close_ch):
        i, j = text.find(open_ch), text.rfind(close_ch)
        return text[i:j+1] if (i != -1 and j != -1 and j > i) else text

    if s.startswith("{"):
        s = _slice_outer(s, "{", "}")
    elif s.startswith("["):
        s = _slice_outer(s, "[", "]")

    return s.strip()


def _looks_like_ipynb(obj) -> bool:
    return isinstance(obj, dict) and {"cells", "metadata", "nbformat"} <= set(obj.keys())


def parse_json_loose(t):
    """
    Try strict, then repaired, then loose slice → strict again.
    Rejects obvious Jupyter notebooks (ipynb) to avoid accidental ingestion.
    """
    raw = t or ""

    # 1) strict
    try:
        d = json.loads(raw)
        if _looks_like_ipynb(d):
            return None
        return d
    except Exception:
        pass

    # 2) repaired
    repaired = _json_repair(raw)
    try:
        d = json.loads(repaired)
        if _looks_like_ipynb(d):
            return None
        return d
    except Exception as e:
        log_exc("outline_loose_repair", e)

    # 3) loose slice of original (first {...} or [...])
    i_obj, j_obj = raw.find("{"), raw.rfind("}")
    i_arr, j_arr = raw.find("["), raw.rfind("]")
    candidates = []
    if i_obj != -1 and j_obj != -1 and j_obj > i_obj:
        candidates.append(raw[i_obj:j_obj+1])
    if i_arr != -1 and j_arr != -1 and j_arr > i_arr:
        candidates.append(raw[i_arr:j_arr+1])
    for cand in candidates:
        try:
            d = json.loads(cand)
            if _looks_like_ipynb(d):
                continue
            return d
        except Exception as e:
            log_exc("outline_loose_slice", e)

    # 4) strict again (last try)
    try:
        d = json.loads(repaired)
        if _looks_like_ipynb(d):
            return None
        return d
    except Exception as e:
        log_exc("outline_loose_raw", e)
        return None

def _fallback_outline(spec):
    """Local minimal outline if the model fails to return valid JSON."""
    n = int((spec or {}).get("chapters", 10) or 10)
    seed = (spec or {}).get("outline_seed", []) or []
    tmpl = (spec or {}).get("chapter_template", {}) or {}
    sections = tmpl.get("sections", []) or [
        "Opening scene",
        "Problem",
        "Small win",
        "Cliffhanger or cozy close",
    ]
    chapters = []
    for i in range(n):
        title = seed[i] if i < len(seed) else f"Chapter {i + 1}"
        chapters.append(
            {
                "number": i + 1,
                "title": str(title)[:80],
                "sections": sections,
                "learning_objectives": [
                    "Enjoy the story",
                    "Notice cause and effect",
                    "Practice empathy",
                ],
            }
        )
    return {"chapters": chapters}

def gen_outline(spec, tr, router):
    pj = "content/outline/outline.json"
    pm = "content/outline/outline.md"
    if has_file(pj) and has_file(pm):
        return pj, pm

    n = int(spec.get("chapters", 10) or 10)
    seed = spec.get("outline_seed", []) or []
    tmpl = (spec.get("chapter_template", {}) or {})
    tmpl_sections = tmpl.get("sections", []) or [
        "Opening scene", "Problem", "Small win", "Cliffhanger or cozy close"
    ]

    instr = (
        'Return JSON ONLY with key "chapters": an array of exactly {n} items.\n'
        "Each chapter object MUST have:\n"
        "- number (1-based integer)\n"
        "- title (<= 7 words; if a matching SeedTitles entry exists, prefer it)\n"
        "- sections (array of short section titles; follow ChapterTemplate sections)\n"
        "- learning_objectives (array of 2-3 kid-friendly bullets)\n"
        "NO markdown fences, NO commentary, NO extra keys. JSON ONLY.\n"
        "Return MINIFIED JSON."
    ).format(n=n)

    max_t = 900 if not pipeline_config.get("ULTRA_BUDGET_MODE") else 500
    cache_key = f"outline:v3:{n}:{sha1('|'.join(seed))}"
    prompt = "BRIEF\n" + brief(spec) + "\n\nINSTRUCTIONS\n" + instr

    # First attempt (JSON mode)
    R = router.author.run(
        "outline",
        prompt,
        max_t=max_t,
        cache_key=cache_key,
        force_json=True
    )
    model_text = R.get("text", "") or ""
    try:
        _probe = json.loads(_json_repair(model_text))
        if _looks_like_ipynb(_probe):
            model_text = ""
    except Exception:
        pass
    
    # Save raw for debugging
    try:
        write_text("logs/outline_model_raw.txt", model_text)
    except Exception:
        pass

    d = parse_json_loose(model_text)

    # One strict retry with stronger “no prose” reminder if needed
    if not (d and isinstance(d.get("chapters"), list) and d["chapters"]):
        strict_prompt = prompt + "\n\nReturn JSON ONLY. Do not include any prose."
        R2 = router.author.run(
            "outline_retry",
            strict_prompt,
            max_t=max_t,
            cache_key=cache_key + ":retry1",
            force_json=True,
        )
        model_text2 = R2.get("text", "") or ""
        try:
            write_text("logs/outline_model_raw_retry1.txt", model_text2)
        except Exception:
            pass
        d = parse_json_loose(model_text2)

    # Fallback to deterministic outline
    if not (d and isinstance(d.get("chapters"), list) and d["chapters"]):
        d = _fallback_outline(spec)
        try:
            write_json(
                "logs/outline_parse_fallback.json",
                {
                    "reason": "Invalid or no JSON from model",
                    "model_text_preview": (model_text or "")[:2000],
                    "ts": now_utc_iso(),
                },
            )
        except Exception:
            pass

    # --- Normalize/repair: exactly n items, numbered 1..n, titles + sections + LOs
    chapters = d.get("chapters", [])
    chapters = (chapters + [{}] * max(0, n - len(chapters)))[:n]

    fixed = []
    for i in range(n):
        ch = chapters[i] if isinstance(chapters[i], dict) else {}
        num = i + 1

        title = (ch.get("title") or "").strip()
        if not title and i < len(seed) and seed[i]:
            title = str(seed[i]).strip()
        if not title:
            title = f"Chapter {num}"
        # keep titles reasonably short (<= 7 words)
        if len(title.split()) > 7:
            title = " ".join(title.split()[:7])

        sections = ch.get("sections") if isinstance(ch.get("sections"), list) else None
        if not sections:
            sections = list(tmpl_sections)

        los = ch.get("learning_objectives") if isinstance(ch.get("learning_objectives"), list) else None
        if not los:
            los = ["Enjoy the story", "Notice cause and effect", "Practice empathy"]
        los = [str(x).strip() for x in los][:3]

        fixed.append(
            {
                "number": num,
                "title": title,
                "sections": sections,
                "learning_objectives": los,
            }
        )

    final = {"chapters": fixed}

    write_json(pj, final)

    # Minimal readable Markdown index
    lines = ["# Outline: " + str(spec.get("title", "")), ""]
    for ch in final["chapters"]:
        lines.append(f"## Chapter {ch['number']}: {ch['title']}")
    write_text(pm, "\n".join(lines))

    return pj, pm

In [9]:
# Cell 7: Per-Chapter Loop (revised)

from pathlib import Path
import json

def ch_dir(ch, sub):
    d = Path(f'content/{sub}/{int(ch):02d}')
    d.mkdir(parents=True, exist_ok=True)
    return d

def outline_ch(pj, ch):
    d = read_json(pj) or {}
    for it in d.get('chapters', []):
        if int(it.get('number', -1)) == int(ch):
            return it
    raise KeyError(f'Chapter {ch} not in outline')

def parse_editor_blocks(t: str):
    """Extract three tagged blocks and return (md, claims_dict, notes_md)."""
    def ex(tag):
        import re
        m = re.search(r'<'+tag+r'>\s*([\s\S]*?)\s*</'+tag+r'>', t, flags=re.DOTALL | re.IGNORECASE)
        return (m.group(1) if m else '').strip()

    md = ex('DRAFT_EDITED_MD')
    claims = ex('CLAIMS_REPORT_JSON')
    notes = ex('CONTINUITY_NOTES_MD')

    # Claims: try JSON parse; otherwise keep raw
    try:
        c = json.loads(sanitize_md(claims)) if claims else {'notes': 'none'}
    except Exception:
        c = {'raw': sanitize_md(claims)}

    return sanitize_md(md), c, sanitize_md(notes)

def process_chapter(ch, cfg, tracker, router, pj):
    ch = int(ch)
    cap = float(cfg.get('CHAPTER_COST_CAP_USD', 0.25))
    start_spent = tracker.spent if tracker else 0.0

    # Chapter directories
    dr = ch_dir(ch, 'research')
    dd = ch_dir(ch, 'drafts')
    de = ch_dir(ch, 'edits')

    # Paths used later (define them unconditionally!)
    b = dr / 'brief.md'
    s = dr / 'sources.json'
    dp = dd / 'draft.md'
    meta_path = dd / '.author_call.json'

    # --- Research (optional) ---
    if cfg.get('RESEARCH_ENABLED'):
        if (not b.exists()) or (not s.exists()):
            oc = outline_ch(pj, ch)
            locals_dir = Path('content/research_inputs') / f'{ch:02d}'
            note = 'locals present' if locals_dir.exists() else 'no locals'
            if router.research:
                prompt = (
                    "OUTLINE_JSON:\n" + json.dumps(oc, ensure_ascii=False) + "\n"
                    "NOTE: " + note + "\n"
                    "Return blocks:\n"
                    "<BRIEF_MD>...</BRIEF_MD>\n"
                    "<SOURCES_JSON>{...}</SOURCES_JSON>\n"
                    "No web."
                )
                R = router.research.run(
                    f'ch{ch}_research', prompt, max_t=500,
                    cache_key=sha1(json.dumps(oc, sort_keys=True) + '|' + note)
                )
                t = R.get('text', '') or ''
            else:
                t = '<BRIEF_MD>Research off</BRIEF_MD><SOURCES_JSON>{"enabled":false}</SOURCES_JSON>'

            import re
            bm = re.search(r'<BRIEF_MD>\s*([\s\S]*?)\s*</BRIEF_MD>', t, flags=re.DOTALL | re.IGNORECASE)
            sm = re.search(r'<SOURCES_JSON>\s*([\s\S]*?)\s*</SOURCES_JSON>', t, flags=re.DOTALL | re.IGNORECASE)
            write_text(b, sanitize_md(bm.group(1)) if bm else '')
            try:
                sj = json.loads(sanitize_md(sm.group(1))) if sm else {'enabled': False}
                write_json(s, sj)
            except Exception as e:
                log_exc(f'ch{ch}_research_sources_parse', e)
                write_json(s, {'raw': sanitize_md(sm.group(1)) if sm else ''})

    # --- Author draft (ALWAYS runs; independent of research) ---
    need_author = (not dp.exists()) or (dp.stat().st_size < 8) or (sanitize_md(read_text(dp)).strip() == '')
    if need_author:
        oc = outline_ch(pj, ch)
        try:
            total = int(book_spec.get('target_length_words', 12000) or 12000)
            n = max(1, int(book_spec.get('chapters', 10) or 10))
            tw = max(450, min(800, total // n))
        except Exception:
            tw = 600

        br = read_text(b) if b.exists() else ''
        prompt = (
            "OUTLINE_JSON:\n" + json.dumps(oc, ensure_ascii=False) + "\n"
            "BRIEF_MD:\n" + (br[:800]) + "\n\n"
            f"Write ONLY Markdown for '## Chapter {ch}: {oc.get('title','')}'.\n"
            "- Use H3 section headings.\n"
            "- Include a short Checklist section.\n"
            "- Add 3 Exercises.\n"
            "- Add [n] where a claim needs a note.\n"
            f"- Target ~{tw} words.\n"
        )

        R = router.author.run(
            f'ch{ch}_draft', prompt, max_t=900,
            cache_key=sha1(json.dumps({'ch': ch, 'oc': oc}, sort_keys=True))
        )
        raw_model = (R.get('text', '') or '')
        write_text(f'logs/ch{ch:02d}_author_raw_MODEL.txt', raw_model)  # NEW: unsanitized
        raw = sanitize_md(R.get('text', '') or '')
        write_text(f'logs/ch{ch:02d}_author_raw.txt', raw)

        write_json(meta_path, {
            "t": now_utc_iso(), "chapter": ch, "outline_title": oc.get("title",""),
            "cached": bool(R.get("cached")), "usage": R.get("usage"),
            "est_cost": R.get("est_cost"), "attempt": "initial"
        })

        if not raw.strip():
            R2 = router.author.run(
                f'ch{ch}_draft_retry1',
                prompt + "\nReturn ONLY Markdown. No commentary.",
                max_t=900,
                cache_key=sha1(json.dumps({'ch': ch, 'oc': oc}, sort_keys=True) + '|retry1|' + now_utc_iso())
            )
            raw = sanitize_md(R2.get('text', '') or '')
            write_text(f'logs/ch{ch:02d}_author_raw_retry1.txt', raw)
            write_json(meta_path, {
                "t": now_utc_iso(), "chapter": ch, "outline_title": oc.get("title",""),
                "cached": bool(R2.get("cached")), "usage": R2.get("usage"),
                "est_cost": R2.get("est_cost"), "attempt": "retry1"
            })

        if not raw.strip():
            raw = (
                f"## Chapter {ch}: {oc.get('title','')}\n\n"
                "### Opening scene\n[TBD]\n\n"
                "### Problem\n[TBD]\n\n"
                "### Small win\n[TBD]\n\n"
                "### Cliffhanger or cozy close\n[TBD]\n"
            )

        write_text(dp, raw)
    else:
        if not meta_path.exists():
            write_json(meta_path, {
                "t": now_utc_iso(), "chapter": ch,
                "note": "Draft existed; AuthorAgent not called this run."
            })

    # --- Ultra-budget: cheap “edit” path ---
    if cfg.get('ULTRA_BUDGET_MODE'):
        ep = de / 'draft_edited.md'
        decp = de / 'claims_report.json'
        nop = de / 'continuity_notes.md'
        draft = read_text(dp)
        cleaned = "\n".join([ln.rstrip() for ln in draft.splitlines()])
        write_text(ep, cleaned)
        write_json(decp, {'info': 'Editor skipped (ULTRA_BUDGET_MODE)', 'chapter': ch})
        write_text(nop, 'Local cleanup applied; no LLM edit due to ultra budget mode.')
        return

    # --- Budget gate for editor ---
    if tracker:
        spent_now = tracker.spent - start_spent
        remaining = tracker.summary().get('remaining_usd', 0.0)
        if (spent_now >= cap) or (remaining < 0.05):
            ep = de / 'draft_edited.md'
            decp = de / 'claims_report.json'
            nop = de / 'continuity_notes.md'
            write_text(ep, read_text(dp))
            write_json(decp, {'warning': 'editor skipped budget', 'chapter': ch})
            write_text(nop, 'Skipped due to budget limits.')
            return

    # --- Editor pass ---
    ep = de / 'draft_edited.md'
    decp = de / 'claims_report.json'
    nop = de / 'continuity_notes.md'
    if not (ep.exists() and decp.exists() and nop.exists()):
        dtext = read_text(dp)
        dtext = dtext if len(dtext) < 12000 else dtext[:12000]
        instr = (
            "INPUT_MD:\n" + dtext + "\n\n"
            "Return EXACTLY these blocks:\n"
            "<DRAFT_EDITED_MD>...</DRAFT_EDITED_MD>\n"
            "<CLAIMS_REPORT_JSON>{\"claims\":[\"...\"]}</CLAIMS_REPORT_JSON>\n"
            "<CONTINUITY_NOTES_MD>...</CONTINUITY_NOTES_MD>\n"
            "Keep the author's style and headings. No URLs."
        )
        R = router.editor.run(f'ch{ch}_edit', instr, max_t=800, cache_key=sha1(dtext))
        md, cl, nt = parse_editor_blocks(R.get('text', '') or '')
        write_text(ep, md or read_text(dp))
        write_json(decp, cl)
        write_text(nop, nt)

print('Chapter processor ready')

Chapter processor ready


In [10]:
# Cell 8: Assembly
from pathlib import Path  # safe even if already imported elsewhere


def assemble_book(spec):
    # Ensure output directory exists
    build_dir = Path("build")
    build_dir.mkdir(parents=True, exist_ok=True)

    # Front matter (defensive: handle missing fields gracefully)
    title = str(spec.get("title", "Untitled")).strip()
    subtitle = str(spec.get("subtitle", "") or "").strip()
    author = str(spec.get("author", "Unknown Author")).strip()

    fm = [
        f"# {title}",
        f"## {subtitle}" if subtitle else "",
        f"**Author:** {author}",
        "",
        "---",
        "",
    ]

    chs, toc = [], []
    n = int(spec.get("chapters", 0))

    for ch in range(1, n + 1):
        e = Path(f"content/edits/{ch:02d}/draft_edited.md")
        d = Path(f"content/drafts/{ch:02d}/draft.md")

        if e.exists():
            t = read_text(e)
            src = "edit"
            p = e
        elif d.exists():
            t = read_text(d)
            src = "draft"
            p = d
        else:
            # FIX: proper newline in placeholder
            t = f"## Chapter {ch}: (missing)\nTODO"
            src = "missing"
            p = None

        chs.append(t.strip())

        # First H2 line as chapter title (fallback to generic)
        tl = next(
            (ln.strip() for ln in t.splitlines() if ln.strip().startswith("## ")),
            f"Chapter {ch}",
        )

        toc.append(
            {
                "chapter": ch,
                "title_line": tl,
                "source": src,
                "path": str(p) if p else None,
            }
        )

    # FIX: correct join with blank line between sections; add trailing newline
    md = "\n\n".join([x for x in fm if x] + chs) + "\n"

    write_text(str(build_dir / "book.md"), md)
    write_json(str(build_dir / "toc.json"), toc)

    return str(build_dir / "book.md"), str(build_dir / "toc.json")


print("Assembly ready")

Assembly ready


In [11]:
# Cell 9: QA
from pathlib import Path
import re


def run_qa(spec):
    dist_dir = Path("dist")
    dist_dir.mkdir(parents=True, exist_ok=True)

    rpt = {"checks": {}, "warnings": [], "metrics": {}}

    book_path = Path("build/book.md")
    ok = book_path.exists() and book_path.stat().st_size > 0
    rpt["checks"]["book_exists"] = bool(ok)
    if not ok:
        write_json(str(dist_dir / "qa_report.json"), rpt)
        return rpt

    t = read_text(book_path)

    # Metrics
    wc = count_words(t)
    rpt["metrics"]["word_count"] = wc

    tgt = int(spec.get("target_length_words", 20000))
    low, hi = int(tgt * 0.9), int(tgt * 1.1)
    within = low <= wc <= hi
    rpt["checks"]["word_count_ok"] = bool(within)
    if not within:
        rpt["warnings"].append(
            f"Word count {wc} vs target {tgt} (acceptable range {low}-{hi})"
        )

    # Chapter presence: prefer TOC if present; otherwise heuristic via H2 count
    n = int(spec.get("chapters", 0))
    toc_path = Path("build/toc.json")
    toc_present = False
    missing_list = []

    if toc_path.exists():
        try:
            toc = read_json(toc_path) or []
            toc_present = (len(toc) == n) and all(
                item.get("source", "unknown") != "missing" for item in toc
            )
            if not toc_present and toc:
                # note which chapters the TOC marked missing (if our Cell 8 added 'source')
                for idx, item in enumerate(toc, start=1):
                    if item.get("source", "unknown") == "missing":
                        missing_list.append(idx)
        except Exception:
            toc_present = False

    if not toc_path.exists() or not toc_present:
        # Fallback: require at least n H2 headings (## ...)
        h2_count = sum(1 for ln in t.splitlines() if ln.strip().startswith("## "))
        toc_present = h2_count >= n
        if not toc_present:
            rpt["warnings"].append(
                f"Expected {n} chapters but found {h2_count} H2 headings."
            )

    rpt["checks"]["all_chapters_present"] = bool(toc_present)
    if missing_list:
        rpt["warnings"].append(f"Chapters marked missing in TOC: {missing_list}")

    # TODO/FIXME markers
    todo_hits = re.findall(r"\b(?:TODO|FIXME)\b", t)
    rpt["checks"]["no_todo_fixme"] = len(todo_hits) == 0
    if todo_hits:
        rpt["warnings"].append(f"TODO/FIXME remain: {len(todo_hits)} occurrences")

    # Heading level sanity: flag H4+ outside code fences (skip ``` blocks)
    def strip_codeblocks(s):
        out, in_code = [], False
        for line in s.splitlines():
            if line.strip().startswith("```"):
                in_code = not in_code
                continue
            if not in_code:
                out.append(line)
        return "\n".join(out)

    text_no_code = strip_codeblocks(t)
    head_ok = "####" not in text_no_code
    rpt["checks"]["heading_levels_ok"] = bool(head_ok)
    if not head_ok:
        rpt["warnings"].append("Headings exceed H3 (#### found)")

    # Citations check when research is enabled for nonfiction
    genre = str(spec.get("genre", "")).lower()
    research_enabled = bool(
        spec.get("research_enabled")
        or (
            isinstance(spec.get("research_policy"), dict)
            and spec["research_policy"].get("enabled")
        )
    )
    cites_ok = True
    if genre == "nonfiction" and research_enabled:
        cites_ok = "[n]" in t
        if not cites_ok:
            rpt["warnings"].append(
                "No [n] markers while research is enabled for nonfiction."
            )
    rpt["checks"]["citations_present_if_research"] = bool(cites_ok)

    # Check end-matter sections per chapter (if template defines them)
    end_matter = (spec.get('chapter_template', {}) or {}).get('end_matter', []) or []
    if end_matter:
        for ch in range(1, int(spec.get('chapters', 10) or 10) + 1):
            block = re.findall(fr"^##\s+Chapter\s+{ch}:[\s\S]*?(?=^##\s+Chapter\s+{ch+1}:|\Z)", t, flags=re.M)
            if block:
                missing = [s for s in end_matter if s not in block[0]]
                if missing:
                    rpt['warnings'].append(f'Ch {ch} missing end-matter: {missing}')

    # Flag very long paragraphs (~>220 words)
    paras = re.split(r'\n\s*\n', t)
    if any(len(p.split()) > 220 for p in paras):
        rpt['warnings'].append('Some paragraphs exceed ~220 words.')

    # Double-space / trailing-space hygiene
    if re.search(r'[^\S\r\n]{2,}', t):
        rpt['warnings'].append('Double spaces detected.')
    if re.search(r'[ \t]+$', t, flags=re.M):
        rpt['warnings'].append('Trailing spaces at line ends.')

    # Claims mismatch: [n] markers vs. claims_report.json presence
    claims_files = list(Path('content/edits').rglob('*/claims_report.json'))
    has_claims_files = any(Path(cf).exists() and Path(cf).stat().st_size > 2 for cf in claims_files)
    has_markers = ('[n]' in t)
    if has_markers and not has_claims_files:
        rpt['warnings'].append('Found [n] markers but no claims_report.json files.')
    if has_claims_files and not has_markers:
        rpt['warnings'].append('claims_report.json exists but no [n] markers found in book.')

    # Outcome
    rpt["outcome"] = "PASS" if not rpt["warnings"] else "PASS_WITH_WARNINGS"

    write_json(str(dist_dir / "qa_report.json"), rpt)
    return rpt


print("QA ready")

QA ready


In [12]:
# Cell 10: Export
from pathlib import Path
import shutil, zipfile, re


def minimal_docx(
    p, note="DOCX export unavailable; install python-docx. See dist/book.md"
):
    p = Path(p)
    p.parent.mkdir(parents=True, exist_ok=True)
    # Basic, valid DOCX container with a single paragraph note
    safe = note.replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
    with zipfile.ZipFile(p, "w", compression=zipfile.ZIP_DEFLATED) as z:
        z.writestr(
            "[Content_Types].xml",
            '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>'
            '<Types xmlns="http://schemas.openxmlformats.org/package/2006/content-types">'
            '<Default Extension="rels" ContentType="application/vnd.openxmlformats-package.relationships+xml"/>'
            '<Default Extension="xml" ContentType="application/xml"/>'
            '<Override PartName="/word/document.xml" ContentType="application/vnd.openxmlformats-officedocument.wordprocessingml.document.main+xml"/>'
            "</Types>",
        )
        z.writestr(
            "_rels/.rels",
            '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>'
            '<Relationships xmlns="http://schemas.openxmlformats.org/package/2006/relationships">'
            '<Relationship Id="rId1" Type="http://schemas.openxmlformats.org/officeDocument/2006/relationships/officeDocument" Target="word/document.xml"/>'
            "</Relationships>",
        )
        z.writestr(
            "word/document.xml",
            '<?xml version="1.0" encoding="UTF-8" standalone="yes"?>'
            '<w:document xmlns:w="http://schemas.openxmlformats.org/wordprocessingml/2006/main">'
            "<w:body><w:p><w:r><w:t>"
            + safe
            + "</w:t></w:r></w:p></w:body></w:document>",
        )


def export_deliverables():
    src = Path("build/book.md")
    dst = Path("dist/book.md")
    dst.parent.mkdir(parents=True, exist_ok=True)

    if src.exists():
        shutil.copyfile(src, dst)
        md = read_text(src)
    else:
        md = "# Book (missing)\n"
        write_text(dst, md)

    # Try rich DOCX via python-docx; otherwise create a minimal DOCX with a note
    try:
        import docx
        from docx.enum.text import WD_BREAK  # page breaks

        d = docx.Document()

        def _sanitize_docx_text(s: str) -> str:
            # Replace tabs with spaces and strip control chars
            s = s.replace("\t", "    ")
            return "".join(ch if ord(ch) >= 0x20 else " " for ch in s)

        in_code = False
        for ln in md.splitlines():
            s = _sanitize_docx_text(ln.rstrip("\n"))

            # Toggle code blocks on fenced markers
            if s.strip().startswith("```"):
                in_code = not in_code
                continue

            if in_code:
                p = d.add_paragraph(s)
                # 'Code' style may not exist; fall back silently
                try:
                    p.style = "Code"
                except Exception:
                    pass
                continue

            if not s.strip():
                d.add_paragraph("")
                continue

            if s.startswith("### "):
                d.add_heading(s[4:], level=3)
            elif s.startswith("## "):
                d.add_heading(s[3:], level=2)
            elif s.startswith("# "):
                d.add_heading(s[2:], level=1)
            elif s.strip() == "---":
                d.add_page_break()
            elif s.lstrip().startswith(("- ", "* ")):
                d.add_paragraph(s.lstrip()[2:], style="List Bullet")
            elif re.match(r"^\s*\d+\.\s+", s):
                d.add_paragraph(re.sub(r"^\s*\d+\.\s+", "", s), style="List Number")
            else:
                d.add_paragraph(s)

        d.save("dist/book.docx")
    except Exception as e:
        minimal_docx(
            "dist/book.docx",
            note=f"DOCX export unavailable ({e.__class__.__name__}). See dist/book.md",
        )

    return str(dst), "dist/book.docx"


print("Export ready")

Export ready


In [13]:
# Cell 11: Manifest
from pathlib import Path
from datetime import datetime, timezone


def manifest(tr, outcome):
    def scan(d):
        root = Path(d)
        items = []
        if not root.exists():
            return items
        for p in sorted(root.rglob("*")):
            if p.is_file():
                st = p.stat()
                items.append(
                    {
                        "path": p.as_posix(),
                        "size_bytes": st.st_size,
                        "mtime": datetime.fromtimestamp(st.st_mtime, timezone.utc)
                        .isoformat()
                        .replace("+00:00", "Z"),
                    }
                )
        return items

    Path("dist").mkdir(parents=True, exist_ok=True)

    m = {
        "outcome": outcome,
        "generated_at": datetime.now(timezone.utc).isoformat().replace("+00:00", "Z"),
        "files": {
            "build": scan("build"),
            "dist": scan("dist"),
        },
        "cost_summary": tr.summary() if hasattr(tr, "summary") else None,
    }

    write_json("dist/manifest.json", m)
    return m


print("Manifest ready")

Manifest ready


In [14]:
# Cell 12: Run-All / Rerun
from pathlib import Path


def make_router():
    tr = CostTracker(pipeline_config["RUN_COST_CAP_USD"])
    rt = PMRouter(book_spec, pipeline_config, tr)
    return rt, tr


def run_all():
    Path("logs").mkdir(parents=True, exist_ok=True)
    if os.getenv('DRY_RUN') == '1':
        print('[run_all] DRY_RUN=1 → skipping API calls; using synthetic outputs.')
    out = "PASS"
    tr = None  # ensure defined for finally
    try:
        rt, tr = make_router()

        # Style & outline
        gen_style(book_spec)
        pj, pm = gen_outline(book_spec, tr, rt)

        # Chapter selection based on config
        total = int(book_spec.get("chapters", 0))
        if pipeline_config.get("FULL_RUN", True):
            chs = list(range(1, total + 1))
        else:
            sample_n = int(pipeline_config.get("SAMPLE_RUN_CHAPTERS", 2))
            chs = list(range(1, min(total, sample_n) + 1))

        # Per-chapter processing with cost-cap handling
        for ch in chs:
            try:
                process_chapter(ch, pipeline_config, tr, rt, pj)
            except CostCapExceededException as e:
                write_text("logs/last_error.txt", str(e))
                out = "ABORTED_COST_CAP"
                break

        # Assembly, QA, Export
        out = "PASS"
        try:
            assemble_book(book_spec)
        except Exception as e:
            log_exc("assemble_book", e)
            return "FAILED_STEP"

        try:
            qa = run_qa(book_spec)
            if qa.get("warnings"):
                out = "PASS_WITH_WARNINGS" if out == "PASS" else out
        except Exception as e:
            log_exc("run_qa", e)
            return "FAILED_STEP"

        try:
            export_deliverables()
        except Exception as e:
            log_exc("export_deliverables", e)
            return "FAILED_STEP"

        return out


    except CostCapExceededException as e:
        write_text("logs/last_error.txt", str(e))
        out = "ABORTED_COST_CAP"
    except Exception as e:
        write_text("logs/last_error.txt", "FAILED_STEP: " + str(e))
        out = "FAILED_STEP"
    finally:
        try:
            manifest(tr, out) if tr else None
        except Exception as e:
            # last-ditch logging if manifest itself fails
            write_text("logs/last_error.txt", "MANIFEST_FAILED: " + str(e))

    print("Run done", out)
    return out


def rerun_chapter(n):
    n = int(n)
    # Clear prior artifacts for this chapter
    for sub in ["research", "drafts", "edits"]:
        d = Path(f"content/{sub}/{n:02d}")
        if d.exists():
            for p in d.glob("*"):
                if p.is_file():
                    p.unlink()

    Path("logs").mkdir(parents=True, exist_ok=True)

    rt, tr = make_router()
    pj = "content/outline/outline.json"
    if not Path(pj).exists():
        pj, _ = gen_outline(book_spec, tr, rt)

    process_chapter(n, pipeline_config, tr, rt, pj)
    assemble_book(book_spec)
    run_qa(book_spec)
    export_deliverables()
    manifest(tr, "PASS")
    print("Rerun done", n)


def resume():
    return run_all()

def verify_author_calls():
    from pathlib import Path, PurePosixPath
    import json
    print("AuthorAgent call verification:\n")
    any_found = False
    # Chapter stamps
    for p in sorted(Path("content/drafts").rglob("*/.author_call.json")):
        any_found = True
        d = json.loads(p.read_text(encoding="utf-8"))
        ch = p.parent.name
        print(f"- drafts/{ch}: called @ {d.get('t')} | cached={d.get('cached')} | title=\"{d.get('outline_title','')}\"")
    if not any_found:
        print("- No .author_call.json stamps found in content/drafts/*")
    print("\nRecent agent call events (tail 8):")
    log = Path("logs/agent_calls.jsonl")
    if log.exists():
        lines = log.read_text(encoding="utf-8").splitlines()[-8:]
        for ln in lines:
            print("  ", ln)
    else:
        print("  logs/agent_calls.jsonl (missing)")
        
print("Controls ready")

Controls ready


In [15]:
# Cell 13: Demonstration Mode
from pathlib import Path
import os, sys, shutil, subprocess

# Cheap demo run: sample a couple of chapters and disable research
pipeline_config["SAMPLE_RUN_CHAPTERS"] = int(pipeline_config.get("SAMPLE_RUN_CHAPTERS", 2))
pipeline_config["FULL_RUN"] = False
pipeline_config["RESEARCH_ENABLED"] = False

try:
    outcome = run_all()
except Exception as e:
    Path("dist").mkdir(parents=True, exist_ok=True)
    manifest = {
        "outcome": "FAILED_STEP",
        "failed_step": "run_all_top_level",
        "error": str(e),
        "trace": traceback.format_exc(),
        "t": datetime.now(timezone.utc).isoformat().replace("+00:00", "Z"),
    }
    Path("dist/manifest.json").write_text(json.dumps(manifest, indent=2), encoding="utf-8")
    print("run_all crashed. Details saved to dist/manifest.json")
    outcome = "FAILED_STEP"

docx_path = Path("dist/book.docx")
pdf_path = Path("dist/book.pdf")
pdf_path.parent.mkdir(parents=True, exist_ok=True)

def _have_exe(cmd: str) -> bool:
    return shutil.which(cmd) is not None

def convert_docx_to_pdf(src: Path, dst: Path) -> bool:
    """
    Best-effort DOCX→PDF:
      1) docx2pdf (Windows/macOS w/ MS Word)
      2) LibreOffice soffice --headless (cross-platform)
      3) pandoc (if installed)
    Returns True on success, False otherwise.
    """
    # 1) docx2pdf on win/darwin only
    try:
        if sys.platform in ("win32", "darwin"):
            from docx2pdf import convert  # may raise ImportError
            print("Converting DOCX to PDF via docx2pdf...")
            convert(str(src), str(dst))
            return dst.exists() and dst.stat().st_size > 0
        else:
            print("docx2pdf is typically not supported on this platform; skipping that route.")
    except Exception as e:
        print(f"docx2pdf failed: {e}")

    # 2) LibreOffice (soffice)
    if _have_exe("soffice"):
        try:
            print("Converting via LibreOffice (soffice --headless)...")
            # LibreOffice writes into output directory; we move/rename if needed
            outdir = dst.parent
            cmd = [
                "soffice", "--headless", "--convert-to", "pdf",
                "--outdir", str(outdir), str(src)
            ]
            subprocess.run(cmd, check=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
            # LibreOffice names file as <name>.pdf in outdir
            produced = outdir / (src.stem + ".pdf")
            if produced.exists():
                if produced != dst:
                    produced.replace(dst)
                return dst.exists() and dst.stat().st_size > 0
        except Exception as e:
            print(f"LibreOffice conversion failed: {e}")

    # 3) Pandoc
    if _have_exe("pandoc"):
        try:
            print("Converting via pandoc...")
            subprocess.run(["pandoc", str(src), "-o", str(dst)], check=True)
            return dst.exists() and dst.stat().st_size > 0
        except Exception as e:
            print(f"pandoc conversion failed: {e}")

    # No available converter
    print("No PDF converter available (docx2pdf/soffice/pandoc not working). Skipping PDF export.")
    return False

if docx_path.exists():
    success = convert_docx_to_pdf(docx_path, pdf_path)
    if success:
        print(f"PDF saved at: {pdf_path}")
    else:
        print("PDF export skipped or failed. See messages above.")
else:
    print("DOCX file not found. Cannot export PDF.")

m = read_json("dist/manifest.json") or {}
print("Outcome:", m.get("outcome", outcome))
print("Cost:", m.get("cost_summary"))

def tree(d):
    d = Path(d)
    if not d.exists():
        print(f"{d.as_posix()} (missing)")
        return
    for p in sorted(d.rglob("*")):
        if p.is_file():
            rel = p.relative_to(d).as_posix()
            print(f"{d.as_posix()}/{rel} ({p.stat().st_size} bytes)")

print("\nBuild:")
tree("build")

print("\nDist:")
tree("dist")

Converting DOCX to PDF via docx2pdf...


  0%|          | 0/1 [00:00<?, ?it/s]

PDF saved at: dist\book.pdf
Outcome: PASS_WITH_WARNINGS
Cost: {'total_spent_usd': 0.003036, 'run_cap_usd': 3.0, 'remaining_usd': 2.996964, 'log_items': 10}

Build:
build/book.md (9115 bytes)
build/toc.json (1292 bytes)

Dist:
dist/book.docx (40907 bytes)
dist/book.md (9115 bytes)
dist/book.pdf (166061 bytes)
dist/manifest.json (950 bytes)
dist/qa_report.json (1609 bytes)


In [16]:
# TRIAGE CELL — diagnose failing step and what's been produced
from pathlib import Path
import json

def _safe_json(path):
    p = Path(path)
    if not p.exists():
        return {}
    try:
        return json.loads(p.read_text(encoding="utf-8"))
    except Exception as e:
        print(f"Couldn't parse {path}: {e}")
        return {}

def _tree(d):
    d = Path(d)
    if not d.exists():
        print(f"{d.as_posix()} (missing)")
        return
    for p in sorted(d.rglob("*")):
        if p.is_file():
            rel = p.relative_to(d).as_posix()
            print(f"{d.as_posix()}/{rel} ({p.stat().st_size} bytes)")

# 1) What the manifest says
man = _safe_json("dist/manifest.json")
print("Outcome:", man.get("outcome"))
print("Failed step:", man.get("failed_step") or man.get("last_step") or man.get("step"))
print("Notes:", man.get("notes"))
print("Artifacts:", man.get("artifacts"))

# 2) Recent logs (names + preview of newest)
logs = sorted(Path("logs").glob("*.json"), key=lambda p: p.stat().st_mtime, reverse=True)
print("\nRecent logs:", [p.name for p in logs[:6]])
if logs:
    lp = logs[0]
    print(f"\nLast log preview: {lp.name}\n")
    txt = lp.read_text(encoding="utf-8", errors="replace")
    print(txt[:2000] + ("..." if len(txt) > 2000 else ""))

# 3) Check if build/book.md or dist/book.docx exist (helps decide whether to re-export)
b = Path("build/book.md")
d = Path("dist/book.docx")
print("\nbuild/book.md exists:", b.exists(), "| size:", (b.stat().st_size if b.exists() else 0))
print("dist/book.docx exists:", d.exists(), "| size:", (d.stat().st_size if d.exists() else 0))

# 4) Quick trees
print("\nBuild tree:")
_tree("build")
print("\nDist tree:")
_tree("dist")

verify_author_calls()


Outcome: PASS_WITH_WARNINGS
Failed step: None
Notes: None
Artifacts: None

Recent logs: []

build/book.md exists: True | size: 9115
dist/book.docx exists: True | size: 40907

Build tree:
build/book.md (9115 bytes)
build/toc.json (1292 bytes)

Dist tree:
dist/book.docx (40907 bytes)
dist/book.md (9115 bytes)
dist/book.pdf (166061 bytes)
dist/manifest.json (950 bytes)
dist/qa_report.json (1609 bytes)
AuthorAgent call verification:

- drafts/01: called @ 2025-09-03T21:29:56.053949Z | cached=False | title="The Whir in the Mango Tree"
- drafts/02: called @ 2025-09-03T21:30:26.323508Z | cached=False | title="A Friend Called Pip"

Recent agent call events (tail 8):
   {"t": "2025-09-03T21:29:43.433164Z", "agent": "Author", "model": "gpt-4o-mini", "label": "ch1_draft", "event": "begin", "cache_key": "0dd4287bfdd9ab12f2fe81e0d3b203e7d6e7e9b1", "max_t": 900, "force_json": false}
   {"t": "2025-09-03T21:29:56.047657Z", "agent": "Author", "model": "gpt-4o-mini", "label": "ch1_draft", "event": "llm